In [63]:
import os
import argparse
import subprocess
import pandas as pd
#from Bio import SeqIO
from shutil import copyfile
from itertools import repeat
from multiprocessing import Pool, freeze_support

In [64]:
InDir = "/mnt/d/Lab/VC_SiLab/data/A3"

In [65]:
r1_end = "_R1_001.fastq.gz"
r2_end = "_R2_001.fastq.gz"

In [66]:
def manifestGen(InDir):
    path = pd.DataFrame()
    for subdir, dirs, files in os.walk(InDir):
        R1 = ""
        R2 = ""
        for file in files:
            if file.endswith(r1_end):
                R1 = os.path.join(subdir, file)
                R2 = os.path.join(subdir, file[:-len(r1_end)]+r2_end)
                sampleStr = file.replace(r1_end, "")
    path = path.append({'SampleID':str(sampleStr), "R1":str(R1), "R2":str(R2)}, ignore_index=True)
    return path

In [67]:
path = manifestGen(InDir)

In [68]:
path

,R1,R2,SampleID
0,/mnt/d/Lab/VC_SiLab/data/A3/A3_S3_L001_R1_001....,/mnt/d/Lab/VC_SiLab/data/A3/A3_S3_L001_R2_001....,A3_S3_L001


In [69]:
df = path

In [70]:
## process manifest
#df = pd.read_table(InFile)
prefixList = df["SampleID"].tolist()
R1List = df["R1"].tolist()
R2List = df["R2"].tolist()

In [71]:
prefixList

['A3_S3_L001']

In [72]:
R1List

['/mnt/d/Lab/VC_SiLab/data/A3/A3_S3_L001_R1_001.fastq.gz']

In [73]:
R2List

['/mnt/d/Lab/VC_SiLab/data/A3/A3_S3_L001_R2_001.fastq.gz']

In [14]:
!fastp -h

option needs value: --html
usage: fastp [options] ... 
options:
  -i, --in1                            read1 input file name (string [=])
  -o, --out1                           read1 output file name (string [=])
  -I, --in2                            read2 input file name (string [=])
  -O, --out2                           read2 output file name (string [=])
      --unpaired1                      for PE input, if read1 passed QC but read2 not, it will be written to unpaired1. Default is to discard it. (string [=])
      --unpaired2                      for PE input, if read2 passed QC but read1 not, it will be written to unpaired2. If --unpaired2 is same as --unpaired1 (default mode), both unpaired reads will be written to this same file. (string [=])
      --failed_out                     specify the file to store reads that cannot pass the filters. (string [=])
  -m, --merge                          for paired-end input, merge each pair of reads into a single read if they are overla

    #" --adapter_fasta /mnt/d/Lab/VC_SiLab/data/adapter.fasta" + \
    #" --cut_tail --cut_window_size 20 --cut_mean_quality 30" + \

    " --length_required 300" + \

In [74]:
## Fastp pair end tirmming and merging
def RunFastp(R1, R2, prefix, OutDir, threads):
    cmd = "fastp --in1 " + R1 + " --in2 " + R2 + \
    " --out1 " + os.path.join(OutDir, prefix + "_R1_unmerged.fastq") + \
    " --out2 " + os.path.join(fastpDir, prefix + "_R2_unmerged.fastq") + \
    " --unpaired1 " + os.path.join(OutDir, prefix + "_R1_unpaired.fastq") + \
    " --unpaired2 " + os.path.join(OutDir, prefix + "_R2_unpaired.fastq") + \
    " --thread " + str(threads) + \
    " --trim_tail1 100 --trim_tail2 100 " + \
    " --merge --merged_out " + os.path.join(OutDir, prefix + ".fastq") + \
    " --correction --overlap_len_require 50" + \
    " --html " + os.path.join(OutDir, prefix + ".html") + \
    " --json " + os.path.join(OutDir, prefix + ".json") + \
    " --report_title " + prefix + "-fastq-merge-report"
    subprocess.call(cmd, shell=True)
    #print(cmd)
## Run fastp in parallel
def RunFastpParallel(R1List, R2List, prefixList, OutDir, threads, jobs):
    pool = Pool(processes = jobs)
    pool.starmap(RunFastp, zip(R1List, R2List, prefixList, repeat(OutDir), repeat(threads)))
    pool.close()
    pool.join()
    pool.terminate()

In [89]:
OutDir = "/mnt/d/Lab/VC_SiLab/result"

In [13]:
OutDir = "/mnt/d/Lab/VC_SiLab/result/demo2_merge"

In [21]:
OutDir = "/mnt/d/Lab/VC_SiLab/result/demo3_merge"

In [75]:
OutDir = "/mnt/d/Lab/VC_SiLab/result/A3"

In [76]:
if os.path.exists(OutDir) == 0:
    os.makedirs(OutDir, 0o777, True)
fastpDir = os.path.join(OutDir, "fastp")
if os.path.exists(fastpDir) == 0:
    os.makedirs(fastpDir, 0o777, True)

In [61]:
fastpDir = "/mnt/d/Lab/VC_SiLab/result/fastp_test"

In [77]:
RunFastpParallel(R1List, R2List, prefixList, fastpDir, 8, 1)

In [ ]:
ERROR: trim_tail1 (--trim_tail1) should be 0 ~ 100, suggest 0 ~ 4

In [59]:
RunFastpParallel(R1List, R2List, prefixList, OutDir, 8, 1)

fastp --in1 /mnt/d/Lab/VC_SiLab/data/A1/A1_S1_L001_R1_001.fastq.gz --in2 /mnt/d/Lab/VC_SiLab/data/A1/A1_S1_L001_R2_001.fastq.gz --out1 /mnt/d/Lab/VC_SiLab/result/A1_S1_L001_R1.fastq --out2 /mnt/d/Lab/VC_SiLab/result/fastp/A1_S1_L001_R2.fastq --thread 8 --cut_tail --cut_window_size 20 --cut_mean_quality 30 --merge --merged_out /mnt/d/Lab/VC_SiLab/result/A1_S1_L001.fastq --correction --overlap_len_require 20 --html /mnt/d/Lab/VC_SiLab/result/A1_S1_L001.html --json /mnt/d/Lab/VC_SiLab/result/A1_S1_L001.json --report_title A1_S1_L001-fastq-merge-report


In [36]:
!ls /mnt/d/Lab/VC_SiLab/result/fastp -alh

total 37M
drwxrwxrwx 1 junyu junyu 4.0K Mar  3 17:25 .
drwxrwxrwx 1 junyu junyu 4.0K Mar  3 17:25 ..
-rwxrwxrwx 1 junyu junyu 452K Mar  3 17:25 A1_S1_L001.html
-rwxrwxrwx 1 junyu junyu 188K Mar  3 17:25 A1_S1_L001.json
-rwxrwxrwx 1 junyu junyu  19M Mar  3 17:25 A1_S1_L001_R1.fastq
-rwxrwxrwx 1 junyu junyu  19M Mar  3 17:25 A1_S1_L001_R2.fastq


部分残余接头序列...? 太少了 无需清楚

末端质量太低，需要双端merge，自动纠错？

In [37]:
from Bio import SeqIO

In [46]:
from Bio.Seq import Seq

In [47]:
seq = Seq("gataccgccccaggacgc")

In [48]:
seq.reverse_complement()

Seq('gcgtcctggggcggtatc')